In [1]:
pip install PyPDF2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 3.5 MB/s eta 0:00:00


In [2]:
pip install pdfminer.six

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 40.0 MB/s eta 0:00:00


In [3]:
pip install pdfplumber

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.5/42.5 kB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.5/59.5 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 55.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 76.1 MB/s eta 0:00:00
  Attempting uninstall: pdfminer.six
    Found existing installation: pdfminer.six 20240706
    Uninstalling pdfminer.six-20240706:
      Successfully uninstalled pdfminer.six-20240706


In [4]:
!sudo apt-get install poppler-utils

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following NEW packages will be installed:
  poppler-utils
0 upgraded, 1 newly installed, 0 to remove and 49 not upgraded.
Need to get 186 kB of archives.
After this operation, 696 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 poppler-utils amd64 22.02.0-2ubuntu0.5 [186 kB]
Fetched 186 kB in 0s (1,145 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 78, <> line 1.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype
dpkg-preconfigure: unable to re-open stdin: 
Selecting previously unselected package poppler-utils.
(Reading database ... 12363

In [5]:
pip install pdf2image

In [6]:
pip install Pillow

In [7]:
!sudo apt install tesseract-ocr
!pip install pytesseract

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  tesseract-ocr-eng tesseract-ocr-osd
The following NEW packages will be installed:
  tesseract-ocr tesseract-ocr-eng tesseract-ocr-osd
0 upgraded, 3 newly installed, 0 to remove and 49 not upgraded.
Need to get 4,816 kB of archives.
After this operation, 15.6 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr-eng all 1:4.00~git30-7274cfa-1.1 [1,591 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr-osd all 1:4.00~git30-7274cfa-1.1 [2,990 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr amd64 4.1.1-2.1build1 [236 kB]
Fetched 4,816 kB in 0s (15.5 MB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debco

In [8]:
# To read the PDF
import PyPDF2
# To analyze the PDF layout and extract text
from pdfminer.high_level import extract_pages, extract_text
from pdfminer.layout import LTTextContainer, LTChar, LTRect, LTFigure
# To extract text from tables in PDF
import pdfplumber
# To extract the images from the PDFs
from PIL import Image
from pdf2image import convert_from_path
# To perform OCR to extract text from images
import pytesseract
# To remove the additional created files
import os

In [9]:
# Create a function to extract text

def text_extraction(element):
    # Extracting the text from the in-line text element
    line_text = element.get_text()

    # Find the formats of the text
    # Initialize the list with all the formats that appeared in the line of text
    line_formats = []
    for text_line in element:
        if isinstance(text_line, LTTextContainer):
            # Iterating through each character in the line of text
            for character in text_line:
                if isinstance(character, LTChar):
                    # Append the font name of the character
                    line_formats.append(character.fontname)
                    # Append the font size of the character
                    line_formats.append(character.size)
    # Find the unique font sizes and names in the line
    format_per_line = list(set(line_formats))

    # Return a tuple with the text in each line along with its format
    return (line_text, format_per_line)

In [10]:
# Create a function to crop the image elements from PDFs
def crop_image(element, pageObj):
    # Get the coordinates to crop the image from the PDF
    [image_left, image_top, image_right, image_bottom] = [element.x0,element.y0,element.x1,element.y1]
    # Crop the page using coordinates (left, bottom, right, top)
    pageObj.mediabox.lower_left = (image_left, image_bottom)
    pageObj.mediabox.upper_right = (image_right, image_top)
    # Save the cropped page to a new PDF
    cropped_pdf_writer = PyPDF2.PdfWriter()
    cropped_pdf_writer.add_page(pageObj)
    # Save the cropped PDF to a new file
    with open('/content/cropped_image.pdf', 'wb') as cropped_pdf_file:
        cropped_pdf_writer.write(cropped_pdf_file)

# Create a function to convert the PDF to images
def convert_to_images(input_file,):
    images = convert_from_path(input_file)
    image = images[0]
    output_file = "/content/PDF_image.png"
    image.save(output_file, "PNG")

# Create a function to read text from images
def image_to_text(image_path):
    # Read the image
    img = Image.open(image_path)
    # Extract the text from the image
    text = pytesseract.image_to_string(img)
    return text

In [11]:
# Extracting tables from the page

def extract_table(pdf_path, page_num, table_num):
    # Open the pdf file
    pdf = pdfplumber.open(pdf_path)
    # Find the examined page
    table_page = pdf.pages[page_num]
    # Extract the appropriate table
    table = table_page.extract_tables()[table_num]
    return table

# Convert table into the appropriate format
def table_converter(table):
    table_string = ''
    # Iterate through each row of the table
    for row_num in range(len(table)):
        row = table[row_num]
        # Remove the line breaker from the wrapped texts
        cleaned_row = [item.replace('\n', ' ') if item is not None and '\n' in item else 'None' if item is None else item for item in row]
        # Convert the table into a string
        table_string+=('|'+'|'.join(cleaned_row)+'|'+'\n')
    # Removing the last line break
    table_string = table_string[:-1]
    return table_string

In [16]:
# Find the PDF path
pdf_path = '/content/Quantum Computing_ Principles and Applications-2.pdf'

# create a PDF file object
pdfFileObj = open(pdf_path, 'rb')
# create a PDF reader object
pdfReaded = PyPDF2.PdfReader(pdfFileObj)

# Create the dictionary to extract text from each image
text_per_page = {}
# We extract the pages from the PDF
for pagenum, page in enumerate(extract_pages(pdf_path)):

    # Initialize the variables needed for the text extraction from the page
    pageObj = pdfReaded.pages[pagenum]
    page_text = []
    line_format = []
    text_from_images = []
    text_from_tables = []
    page_content = []
    # Initialize the number of the examined tables
    table_num = 0
    first_element= True
    table_extraction_flag= False
    # Open the pdf file
    pdf = pdfplumber.open(pdf_path)
    # Find the examined page
    page_tables = pdf.pages[pagenum]
    # Find the number of tables on the page
    tables = page_tables.find_tables()


    # Find all the elements
    page_elements = [(element.y1, element) for element in page._objs]
    # Sort all the elements as they appear in the page
    page_elements.sort(key=lambda a: a[0], reverse=True)

    # Find the elements that composed a page
    for i,component in enumerate(page_elements):
        # Extract the position of the top side of the element in the PDF
        pos= component[0]
        # Extract the element of the page layout
        element = component[1]

        # Check if the element is a text element
        if isinstance(element, LTTextContainer):
            # Check if the text appeared in a table
            if table_extraction_flag == False:
                # Use the function to extract the text and format for each text element
                (line_text, format_per_line) = text_extraction(element)
                # Append the text of each line to the page text
                page_text.append(line_text)
                # Append the format for each line containing text
                line_format.append(format_per_line)
                page_content.append(line_text)
            else:
                # Omit the text that appeared in a table
                pass

        # Check the elements for images
        if isinstance(element, LTFigure):
            # Crop the image from the PDF
            crop_image(element, pageObj)
            # Convert the cropped pdf to an image
            convert_to_images('/content/cropped_image.pdf')
            # Extract the text from the image
            image_text = image_to_text('/content/PDF_image.png')
            text_from_images.append(image_text)
            page_content.append(image_text)
            # Add a placeholder in the text and format lists
            page_text.append('image')
            line_format.append('image')

        # Check the elements for tables
        if isinstance(element, LTRect):
            # If the first rectangular element
            if first_element == True and (table_num+1) <= len(tables):
                # Find the bounding box of the table
                lower_side = page.bbox[3] - tables[table_num].bbox[3]
                upper_side = element.y1
                # Extract the information from the table
                table = extract_table(pdf_path, pagenum, table_num)
                # Convert the table information in structured string format
                table_string = table_converter(table)
                # Append the table string into a list
                text_from_tables.append(table_string)
                page_content.append(table_string)
                # Set the flag as True to avoid the content again
                table_extraction_flag = True
                # Make it another element
                first_element = False
                # Add a placeholder in the text and format lists
                page_text.append('table')
                line_format.append('table')

            # Check if we already extracted the tables from the page
            if element.y0 >= lower_side and element.y1 <= upper_side:
                pass
            elif not isinstance(page_elements[i+1][1], LTRect):
                table_extraction_flag = False
                first_element = True
                table_num+=1


    # Create the key of the dictionary
    dctkey = 'Page_'+str(pagenum)
    # Add the list of list as the value of the page key
    text_per_page[dctkey]= [page_text, line_format, text_from_images,text_from_tables, page_content]

# Closing the pdf file object
pdfFileObj.close()

# Deleting the additional files created


# print(pagenum)
# Display the content of the page
for i in range(pagenum):
  result = ''.join(text_per_page[f'Page_{i}'][3])
  print(result)








|𝜓 𝑖𝑛1|𝜓 𝑖𝑛2|𝜓 𝑜𝑢𝑡1|𝜓 𝑜𝑢𝑡2|
|0|0|0|0|
|0|1|0|1|
|1|0|1|1|
|1|1|1|0|
|𝐼𝑁 1|𝐼𝑁 2|𝑂𝑢𝑡|
|0|0|1|
|0|1|1|
|1|0|1|
|1|1|0||𝜓 𝑖𝑛1|𝜓 𝑖𝑛2|𝜓 𝑖𝑛3|𝜓 𝑜𝑢𝑡1|𝜓 𝑜𝑢𝑡2|𝜓 𝑜𝑢𝑡3|
|0|0|1|0|0|1|
|0|1|1|0|1|1|
|1|0|1|1|0|1|
|1|1|1|1|1|0|
|𝜓 𝑖𝑛1|None|None|𝜓 𝑖𝑛2|None|None|𝜓 𝑖𝑛3|None|None|𝜓 𝑜𝑢𝑡1|None|None|𝜓 𝑜𝑢𝑡2|None|None|𝜓 𝑜𝑢𝑡3|None|None|
|0|None|None|0|None|None|0|None|None|0|None|None|0|None|None|0|None|None|
||0|||0|||1|||0|||0|||1||
|0|None|None|1|None|None|0|None|None|0|None|None|1|None|None|0|None|None|
||0|||1|||1|||0|||1|||1||
|1|None|None|0|None|None|0|None|None|1|None|None|0|None|None|0|None|None|
||1|||0|||1|||1|||0|||1||
|1|None|None|1|None|None|0|None|None|1|None|None|1|None|None|1|None|None|
||1|||1|||1|||1|||1|||0||










||Alice|None|None|Bob|None|None|
|Chosen Base|Random Bit r i|Rotation Angle|Photon State|Chosen Base|Photon State after PBS|Measured Result|
|Horizontal -Vertical (H-V)|0|0°|↔|H-V|↔|0|
|None|None|None|None|D|⤢ or ⤡|?|
|None|1|90°|↕|H-V|↕|1|
|None|None|None|None|D|⤢ or ⤡

In [21]:
from pdfminer.pdfpage import PDFPage
from pdfminer.pdftypes import PDFObjRef, resolve1
from pdfminer.psparser import PSLiteral

pdf_file = '/content/Quantum Computing_ Principles and Applications-2.pdf'

def resolve_dest(doc, dest):
    if isinstance(dest, str):
        dest = resolve1(doc.get_dest(dest))
    elif isinstance(dest, PSLiteral):
        dest = resolve1(doc.get_dest(dest.name))
    if isinstance(dest, dict):
        dest = dest['D']
    if isinstance(dest, PDFObjRef):
        dest = dest.resolve()
    return dest

with pdfplumber.open(pdf_file) as pdf:
    pdf_doc = pdf.doc
    outlines = pdf_doc.get_outlines()

    pages = {page.pageid: pageno for (pageno, page)
             in enumerate(PDFPage.create_pages(pdf_doc), 1)}

    for (level, title, dest, a, se) in outlines:
        pageno = None
        if dest:
            dest = resolve_dest(pdf_doc, dest)
            pageno = pages[dest[0].objid]
        elif a:
            action = a
            if isinstance(action, dict):
                subtype = action.get('S')
                if subtype and repr(subtype) == '/\'GoTo\'' and action.get(
                        'D'):
                    dest = resolve_dest(action['D'])
                    pageno = pages[dest[0].objid]
        print(f'level={level}, title={title}')

level=1, title=Quantum Computing: Principles and Applications
level=2, title=Recommended Citation
level=1, title=Keywords: Quantum computer, Quantum gate, QKD, Shor, Grover


In [ ]:
import pdfplumber

def extract_text_with_layout(pdf_path):
    with pdfplumber.open(pdf_path) as pdf:
        for page in pdf.pages:
            # Extract text with layout
            text = page.extract_text(layout=True)
            print("Full Text with Layout:\n", text)

            # Get detailed character data
            char_data = page.chars
            headings = []
            paragraphs = []
            current_paragraph = []

            for char in char_data:
                text = char['text']
                font_size = char['size']
                is_bold = 'Bold' in char.get('fontname', '')

                # Use font size or boldness as heading indicator
                if font_size > 12 or is_bold:  # Adjust threshold based on your PDF
                    if current_paragraph:
                        paragraphs.append(''.join(current_paragraph))
                        current_paragraph = []
                    headings.append(text)
                else:
                    current_paragraph.append(text)

            if current_paragraph:
                paragraphs.append(''.join(current_paragraph))

            print("Headings:", headings)
            print("Paragraphs:", paragraphs)

# Replace with your PDF file path
pdf_path = "/content/Quantum Computing_ Principles and Applications-2.pdf"
extract_text_with_layout(pdf_path)
